### Final Project. Due Friday, Dec 15 by midnight.

---

Several of the questions build on each other. It might be beneficial to write some views for those so you don't end up duplicating the same CTEs/subqueries.

Thanks for a great semester!

---

In [99]:
import store as store
import sqlite3
from importlib import reload
reload(store)
import dbga
import pandas as pd
from collections import defaultdict as dd
import json

In [100]:
conn = sqlite3.connect('databases/store.db')
curs = conn.cursor()

In [101]:
my_sql = dd(dict)
my_sql['setup'] = []

In [102]:
sql_date_drop = 'DROP VIEW IF EXISTS vOrder_DatesParsed;'
my_sql['setup'].append(sql_date_drop)
curs.execute(sql_date_drop)

sql_date = '''
CREATE VIEW vOrder_DatesParsed as
SELECT *, SUBSTR(date, 1, 4) as year,
          SUBSTR(date, 6, 2) as month,
          SUBSTR(date, 9, 2) as day,
          SUBSTR(date, 1, 7) as year_month
FROM tOrder
;'''
my_sql['setup'].append(sql_date)
curs.execute(sql_date)

In [103]:
pd.read_sql('SELECT * FROM vOrder_DatesParsed', conn)

,order_id,cust_id,date,year,month,day,year_month
0,1,1,2019-01-01,2019,01,01,2019-01
1,2,2,2019-01-02,2019,01,02,2019-01
2,3,3,2019-01-02,2019,01,02,2019-01
3,4,4,2019-01-02,2019,01,02,2019-01
4,5,5,2019-01-02,2019,01,02,2019-01
...,...,...,...,...,...,...,...
2922,2923,75,2021-10-29,2021,10,29,2021-10
2923,2924,23,2021-10-29,2021,10,29,2021-10
2924,2925,222,2021-10-30,2021,10,30,2021-10
2925,2926,18,2021-10-31,2021,10,31,2021-10


In [104]:
#Creating a summary of sales/orders by month for the following questions
sql_sale_drop = 'DROP VIEW IF EXISTS vSales_Summary;'
my_sql['setup'].append(sql_sale_drop)
curs.execute(sql_sale_drop)

sql_sale = '''
CREATE VIEW vSales_Summary as
WITH
Columns as
(
    SELECT year_month, 
           sum(qty*unit_price) as TotalSales, 
           count(DISTINCT order_id) as NumOrders, 
           count(DISTINCT cust_id) as NumCust
    FROM vOrder_DatesParsed
    JOIN tOrderDetail USING(order_id)
    JOIN tProd USING(prod_id)
    GROUP BY year_month
)

SELECT year_month,
       TotalSales,
       NumOrders,
       NumCust,
       1.*TotalSales/NumCust as SalesPerCust,
       1.*TotalSales/NumOrders as SalesPerOrder
FROM Columns
GROUP BY year_month
ORDER BY year_month ASC
;'''

my_sql['setup'].append(sql_sale)
curs.execute(sql_sale)

In [105]:
sql_prod_drop = 'DROP VIEW IF EXISTS vProd_Summary;'
my_sql['setup'].append(sql_prod_drop)
curs.execute(sql_prod_drop)

sql_prod = '''
CREATE VIEW vProd_Summary as
With
Columns as

(
    SELECT year_month, prod_id, prod_desc, unit_price, IFNULL(sum(tOrderDetail.qty), 0) as TotalQty
    FROM vOrder_DatesParsed
    LEFT JOIN tOrderDetail USING(order_id)
    LEFT JOIN tProd USING(prod_id)
    GROUP BY prod_id, year_month
)

SELECT *, IFNULL(sum(TotalQty*unit_price), 0) as TotalSales
FROM Columns
GROUP BY prod_id, year_month
ORDER BY prod_id ASC
;'''

my_sql['setup'].append(sql_prod)
curs.execute(sql_prod)

---
1) Generate a summary, by month and year of how our store is performing.

Have your query return the following:
 - year_month (as a single column, like YYYY-MM)
 - Sales: total sales for the month (i.e., sum of qty * unit price)
 - NumOrders: number of orders placed for the month
 - NumCust: number of _distinct_ customers who made a purchase (i.e. only count the customer at most once per month)
 - OrdersPerCust: average number of orders per customer (i.e. NumOrders/NumCust)
 - SalesPerCust: average sales per customer (i.e. Sales/NumCust)
 - SalesPerOrder: average sales per order (i.e. Sales/NumOrders)

Sort the results should by year_month, in ascending order.

_Hint: Watch out for integer division!_

In [106]:
sql1 = '''
SELECT * 
FROM vSales_Summary
;'''
pd.read_sql(sql1, conn)

,year_month,TotalSales,NumOrders,NumCust,SalesPerCust,SalesPerOrder
0,2019-01,68464.61,91,85,805.466000,752.358352
1,2019-02,55560.32,80,73,761.100274,694.504000
2,2019-03,19191.68,51,49,391.666939,376.307451
3,2019-04,20912.07,48,46,454.610217,435.668125
4,2019-05,11973.34,50,46,260.290000,239.466800
5,2019-06,13737.30,43,41,335.056098,319.472093
6,2019-07,22095.05,45,41,538.903659,491.001111
7,2019-08,15675.05,51,49,319.898980,307.353922
8,2019-09,9360.38,40,39,240.009744,234.009500
9,2019-10,48411.35,58,51,949.242157,834.678448


In [107]:
my_sql['q1']['sql'] = sql1

---
2) In which month did we have the lowest total sales?

Return one record with:
- year_month (as a single column, like YYYY-MM)
- sales (sum of qty * unit_price)

In [108]:
sql2 = '''
SELECT year_month, MIN(TotalSales)
FROM vSales_Summary
;'''
pd.read_sql(sql2, conn)

,year_month,MIN(TotalSales)
0,2019-09,9360.38


In [109]:
my_sql['q2']['sql'] = sql2

---

3. In the month determined from the previous question, generate a list of our total sales by state. Make sure that all states are included, even if they have no sales (50 states + PR and DC = 52 total records). For states having 0 sales, return 0, not NULL or NaN.

Return:

- state_id
- Total sales for the month in question

Order the results by the state_id, ascending

In [110]:
sql3 = '''
WITH
LowSaleMonth as
(
    SELECT year_month, MIN(TotalSales) as MinSales
    FROM vSales_Summary
    
)

SELECT state_id, IFNULL(SUM(qty * unit_price), 0) AS total_sales
FROM tState 
LEFT JOIN tZip USING(state_id)
LEFT JOIN tCust USING(zip)
LEFT JOIN vOrder_DatesParsed ON tCust.cust_id = vOrder_DatesParsed.cust_id AND vOrder_DatesParsed.year_month = (SELECT year_month FROM LowSaleMonth)
LEFT JOIN tOrderDetail USING(order_id)
LEFT JOIN tProd USING(prod_id)

GROUP BY state_id
ORDER BY state_id ASC
;'''
pd.read_sql(sql3, conn)

,state_id,total_sales
0,AK,0.00
1,AL,28.00
2,AR,0.00
3,AZ,0.00
4,CA,557.00
5,CO,0.00
6,CT,0.00
7,DC,0.00
8,DE,0.00
9,FL,414.00


In [111]:
my_sql['q3']['sql'] = sql3

---

4. For the list of states above that had 0 sales, generate a list of all the customers in those states, along with how much they have bought from us since then.

Return:
- cust_id
- first
- last
- addr
- city
- st
- zip
- the customer's total sales from all months after the month from question 4

Order the results by cust_id, ascending

In [112]:
sql4 = '''
WITH
LowSaleMonth as
(
    SELECT year_month, MIN(TotalSales) as MinSales
    FROM vSales_Summary
    
),

Sales_By_Month_State as
(
    SELECT state_id, IFNULL(SUM(qty * unit_price), 0) AS total_sales
    FROM tState 
    LEFT JOIN tZip USING(state_id)
    LEFT JOIN tCust USING(zip)
    LEFT JOIN vOrder_DatesParsed ON tCust.cust_id = vOrder_DatesParsed.cust_id AND vOrder_DatesParsed.year_month = (SELECT year_month FROM LowSaleMonth)
    LEFT JOIN tOrderDetail USING(order_id)
    LEFT JOIN tProd USING(prod_id)
    GROUP BY state_id
    HAVING total_sales = 0
    ORDER BY state_id ASC
),

No_Sales_states as
(
SELECT cust_id, state_id
FROM tCust
JOIN tZip USING(zip)
WHERE state_id IN (SELECT state_id FROM Sales_By_Month_State)
),

Not_Really_Sure as
(
SELECT *
FROM vOrder_DatesParsed
WHERE year_month > (SELECT year_month FROM LowSaleMonth)
)

SELECT cust_id, first, last, addr, city, state_id, zip, SUM(qty * unit_price) as Total_Sales
FROM tCust
JOIN tZip USING(zip)
JOIN Not_Really_Sure USING(cust_id)
JOIN tOrderDetail USING(order_id)
JOIN tProd USING(prod_id)
WHERE state_id IN (SELECT state_id FROM No_Sales_States)
GROUP BY cust_id
ORDER BY cust_id ASC
;'''
pd.read_sql(sql4, conn)

,cust_id,first,last,addr,city,state_id,zip,Total_Sales
0,3,Dodonna,Garza,3639 Briarwood Court,Tarzan,TX,79783,364.90
1,6,Motti,Hunt,1919 Smith Street,Hamden,CT,06514,5465.67
2,7,Mace Windu,Woodward,5655 Jefferson Court,Milan,NM,87021,9422.33
3,11,Senator Ask Aak,Chen,2027 Pheasant Run,Wayne,NJ,07470,11028.03
4,12,Jabba,Oneill,9371 Walnut Street,Swampscott,MA,01907,9848.63
...,...,...,...,...,...,...,...,...
113,297,Tey How,Rodriguez,9164 Madison Street,Brick,NJ,08723,4347.75
114,301,Rabe,Mitchell,8021 Heather Lane,Pindall,AR,72669,8828.98
115,302,Taun We,Miller,4011 Sycamore Drive,Yellow Jacket,CO,81335,11850.42
116,305,Dodonna,Francis,8070 Briarwood Court,Buckland,AK,99727,4878.76


In [113]:
my_sql['q4']['sql'] = sql4

---

5) Get a list of customers who did not purchase anything in the most recent month of data, along with their average sales for all months prior.

Return:

- customer id
- name, address, zip, city, st (abbreviation is fine)
- average sales for all months prior (divide by N-1, where N is the total number of months in the database)

Order the results with the largest average monthly sales on top.

In [114]:
sql5 = '''
With
TotalMonths as
( 
    SELECT count(DISTINCT year_month)
    FROM vSales_Summary
),

MostRecentMonth as
(
    SELECT year_month
    FROM vOrder_DatesParsed
    ORDER BY year_month DESC
    LIMIT 1
),

RecentMonthOrders as
(
    SELECT * 
    FROM vOrder_DatesParsed
    WHERE year_month = (SELECT year_month FROM MostRecentMonth)
)

SELECT DISTINCT cust_id, first, last, zip, city, state_id, sum(qty*unit_price)/((SELECT * FROM TotalMonths)-1) as AvgPriorMonthSales
FROM tCust
JOIN tZip USING(zip)
JOIN vOrder_DatesParsed USING(cust_id)
JOIN tOrderDetail USING(order_id)
JOIN tProd USING(prod_id)
WHERE cust_id NOT IN (SELECT cust_id FROM RecentMonthOrders)
GROUP BY cust_id
ORDER BY AvgPriorMonthSales DESC
;'''
pd.read_sql(sql5, conn)

,cust_id,first,last,zip,city,state_id,AvgPriorMonthSales
0,39,Unkar Plutt,Woodward,51650,Riverton,IA,590.661212
1,46,Captain Antilles,Walker,20005,Washington,DC,561.206667
2,152,Unkar Plutt,Schmidt,13623,Chippewa Bay,NY,533.290000
3,207,Jobal,Mitchell,38629,Falkner,MS,519.972424
4,202,Gold Leader,Elliott,25938,Victor,WV,509.008182
...,...,...,...,...,...,...,...
178,142,Mace Windu,Greene,61957,Windsor,IL,43.382121
179,243,Darth Maul,Walters,58046,Hope,ND,39.101212
180,241,Bala-Tik,Zhang,93673,Traver,CA,36.436364
181,233,Clone Commander Cody,Benson,28390,Spring Lake,NC,25.335152


In [115]:
my_sql['q5']['sql'] = sql5

---
6) Are there any products we haven't sold at least 1 of each month?

If so, return:
 
- prod_id
- prod_desc
- unit_price
- year_month

Order the results by prod_id, year_month, both ascending

In [116]:
sql6 = '''
With
All_Months as
( 
    SELECT year_month
    FROM vSales_Summary
),

All_Products as
(
    SELECT prod_id, prod_desc, unit_price
    FROM tProd
),

All_Combos as
(
    SELECT *
    FROM All_Months
    CROSS JOIN All_Products
)

SELECT prod_id, prod_desc, unit_price, year_month
FROM All_Combos
WHERE (All_Combos.year_month || All_Combos.prod_id) NOT IN (SELECT vProd_Summary.year_month || vProd_Summary.prod_id FROM vProd_Summary)
ORDER BY prod_id ASC, year_month ASC

;'''
pd.read_sql(sql6, conn)

,prod_id,prod_desc,unit_price,year_month
0,300,Washer,0.10,2019-09
1,301,Nail,0.25,2019-09
2,305,Bradawl,1.99,2019-09
3,307,Sandpaper,3.00,2019-03
4,308,Screwdriver,3.00,2019-05
5,309,Chisel,4.99,2019-07
6,310,Scraper,7.99,2019-04
7,312,Plane,10.99,2019-07
8,313,Wrench,11.00,2019-06
9,314,Mallet,12.00,2019-06


In [117]:
my_sql['q6']['sql'] = sql6

---
7) What are our top 5 selling products (in terms of total dollars sold)?

Return:

- prod_id
- prod_desc
- unit_price
- total sales (i.e. sum of qty * unit price)

Order the results by total sales, descending

In [118]:
sql7 = '''
SELECT prod_id, prod_desc, unit_price, TotalSales
FROM vProd_Summary
GROUP BY prod_id
ORDER BY TotalSales DESC
LIMIT 5
;'''
pd.read_sql(sql7, conn)

,prod_id,prod_desc,unit_price,TotalSales
0,329,Chainsaw,499.99,25999.48
1,328,Workbench,300.00,14400.00
2,326,Drill,69.00,5313.00
3,325,Toolbox,50.00,2700.00
4,322,Monkeywrench,29.95,2485.85


In [119]:
my_sql['q7']['sql'] = sql7

---

8) What month did we have our highest sales, and what was our top selling product that month? (All in terms of dollars).

Return:

- year_month
- prod_id
- prod_desc
- The total sales for that month, for all products
- The total sales for that month, for the top selling product only
- The total quantity for that month, for the top selling product only

In [120]:
sql8 = '''
With
Highest_Selling_Month as
(
    SELECT year_month, MAX(TotalSales) as TotalSales
    FROM vSales_Summary
),

Top_Selling_Product as
(
    SELECT year_month, prod_id, prod_desc, TotalQty, TotalSales
    FROM vProd_Summary
    WHERE year_month LIKE (SELECT year_month FROM Highest_Selling_Month)
    ORDER BY TotalSales DESC
    LIMIT 1
)

SELECT Highest_Selling_Month.year_month, prod_id, prod_desc, 
                Highest_Selling_Month.TotalSales as AllProdSales,
                Top_Selling_Product.TotalSales as TopProdSales,
                Top_Selling_Product.TotalQty
FROM Highest_Selling_Month
JOIN Top_Selling_Product
;'''
pd.read_sql(sql8, conn)                   

,year_month,prod_id,prod_desc,AllProdSales,TopProdSales,TotalQty
0,2021-10,329,Chainsaw,251768.12,86998.26,174


In [121]:
my_sql['q8']['sql'] = sql8

In [122]:
with open('Final.data302', 'w') as f:
    json.dump(my_sql, f)

In [123]:
dbga.validate_submission_format(path_db = 'databases/store.db',
                                path_submission = 'Final.data302',
                                num_questions = 8)

Report saved to:  VALIDATION_Final.txt


In [ ]:
db.close()